# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)

__Hi, everyone!!!__

*My name is *Kevin Torres* and I just wanna say:*

*Thanks for watching my notebook*  :)

### *In this notebook I will try to make a ****Recommender System****. *

#### First we have to import some of the libraries that will help us

In [75]:
# importing libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#!conda install -c conda-forge geopy --yes 
import geopy.geocoders

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries are imported.')

Libraries are imported.


#### Now I'm going to load the dataset that I used in week 3 to use the Zip Codes

In [76]:
zct = pd.read_csv('toronto.csv')
zct.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Let's make this more visual by creating a map

In [77]:
# I extracted the latitude and longitude from a website
# https://www.latlong.net/place/toronto-city-hall-toronto-on-canada-11756.html
t_lat = 43.653908
t_lon = -79.384293
t_map = folium.Map(location = [t_lat, t_lon], zoom_start = 10.7)

for lat, lng, borough, neighborhood in zip(zct['Latitude'], zct['Longitude'], zct['Borough'], zct['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#adff2f',
        fill_opacity=0.7).add_to(t_map)  
    

t_map

#### From now on, and to make this easier, I'm going to take a look at the 'Boroughs' and focus on just one of them

In [78]:
#zct['Borough']
zct['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'West Toronto', 'York',
       'Etobicoke'], dtype=object)

#### Once I saw the Boroughs on my dataset I'm going to stay with '*__North York__*'

In [79]:
north_york = zct[zct['Borough'] == 'North York']
north_york = north_york.reset_index(drop=True)
north_york.index = north_york.index + 1
north_york.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M2J,North York,Henry Farm,43.778517,-79.346556
3,M2K,North York,Bayview Village,43.786947,-79.385975
4,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
5,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


#### As I previously made, I'm going to make a map from ***North York***

In [80]:
ny_borough = 'North York, Toronto'

# To get the latitude and longitude I used this website https://www.latlong.net/place/north-york-toronto-on-canada-1373.html
ny_lat = 43.761539
ny_lon = -79.411079
print('The geograpical coordinates of "North York" are: {}, {}.'.format(ny_lat, ny_lon))

ny_map = folium.Map(location=[ny_lat, ny_lon], zoom_start=11.5)

for lat, lng, label in zip(north_york['Latitude'], north_york['Longitude'], north_york['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#1e90ff',
        fill_opacity = 0.7).add_to(ny_map)  
    
ny_map

The geograpical coordinates of "North York" are: 43.761539, -79.411079.


In [81]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [82]:
CLIENT_ID = 'RCFWSM2GUA5GYIVMBDLG2IXPITJNCRSBMQUPPCHQEHWBLVDR'
CLIENT_SECRET = '0Y1I1TWMMBKTAVWBZX04S45JNMO0U0LO0YWH0A4BL2GUXUHD'
VERSION = '20180604' 

#### On the next steps I'm going to take a look at venues in neighborhoods inside ***North York***

In [83]:
ny_foursquare = foursquare_crawler(list(north_york['Postcode']),
                                                   list(north_york['Neighbourhood']),
                                                   list(north_york['Latitude']),
                                                   list(north_york['Longitude']),)

1.
Data is Obtained, for the Postal Code M2H (and Neighborhoods Hillcrest Village) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M2J (and Neighborhoods Henry Farm) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M2K (and Neighborhoods Bayview Village) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M2L (and Neighborhoods Silver Hills, York Mills) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M2M (and Neighborhoods Newtonbrook, Willowdale) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M2R (and Neighborhoods Willowdale West) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M3A (and Neighborhoods Parkwoods) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M3C (and Neighborhoods Flemingdon Park) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M3H (and Neighborhoods Bathurst Manor, Wilson Heights) SUCCESSFULLY.
10.
Data is Obtained, for the Postal Code M3J (and Neighborhoods Northwood Park, York University) SUCCESSFULLY.
11.
Data i

#### To make more easier the next steps, I'm going to save the previous results 

In [84]:
import pickle
with open("ny_foursquare.txt", "wb") as fp:
    pickle.dump(ny_foursquare, fp)
print('Congrats!!! You have saved the data you previously obtained!')

Congrats!!! You have saved the data you previously obtained!


In [85]:
with open("ny_foursquare.txt", "rb") as fp:
    ny_foursquare = pickle.load(fp)
print(type(ny_foursquare))

<class 'list'>


#### It's very important to have accurate data to perform a good analysis, for that reason, cleaning the data is a must

The next code is going to extract the data for each venue on every neighborhood

In [86]:
def get_venue_dataset(foursquare_dataset):
    ny_clean = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venues in Coordination "{}" Postal Code and "{}" Neighborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
                    
            ny_clean = ny_clean.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
    
    return(ny_clean)


In [87]:
ny_venues = get_venue_dataset(ny_foursquare)

Number of Venues in Coordination "M2H" Postal Code and "Hillcrest Village" Neighborhood(s) is:
20
Number of Venues in Coordination "M2J" Postal Code and "Henry Farm" Neighborhood(s) is:
42
Number of Venues in Coordination "M2K" Postal Code and "Bayview Village" Neighborhood(s) is:
13
Number of Venues in Coordination "M2L" Postal Code and "Silver Hills, York Mills" Neighborhood(s) is:
4
Number of Venues in Coordination "M2M" Postal Code and "Newtonbrook, Willowdale" Neighborhood(s) is:
36
Number of Venues in Coordination "M2R" Postal Code and "Willowdale West" Neighborhood(s) is:
11
Number of Venues in Coordination "M3A" Postal Code and "Parkwoods" Neighborhood(s) is:
29
Number of Venues in Coordination "M3C" Postal Code and "Flemingdon Park" Neighborhood(s) is:
37
Number of Venues in Coordination "M3H" Postal Code and "Bathurst Manor, Wilson Heights" Neighborhood(s) is:
27
Number of Venues in Coordination "M3J" Postal Code and "Northwood Park, York University" Neighborhood(s) is:
18
Nu

#### Let's take a look on the venues

In [88]:
ny_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M2H,Hillcrest Village,43.803762,-79.363452,Tastee,This spot is popular,Bakery,692
1,M2H,Hillcrest Village,43.803762,-79.363452,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,This spot is popular,Korean Restaurant,754
2,M2H,Hillcrest Village,43.803762,-79.363452,Galati,This spot is popular,Grocery Store,815
3,M2H,Hillcrest Village,43.803762,-79.363452,Cummer Park,This spot is popular,Park,776
4,M2H,Hillcrest Village,43.803762,-79.363452,Tim Hortons,This spot is popular,Coffee Shop,731


In [89]:
ny_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
431,M9M,"Emery, Humberlea",43.724766,-79.532242,Fiera Foods,This spot is popular,Bakery,823
432,M9M,"Emery, Humberlea",43.724766,-79.532242,Hwy 401 & Hwy 400,This spot is popular,Intersection,875
433,M9M,"Emery, Humberlea",43.724766,-79.532242,Joseph Bannon Park,This spot is popular,Park,889
434,M9M,"Emery, Humberlea",43.724766,-79.532242,Multimodal Website Marketing,This spot is popular,Business Service,924
435,M9M,"Emery, Humberlea",43.724766,-79.532242,U-Haul at Weston Rd,This spot is popular,Storage Facility,930


#### Once I've done all this cleaning, it's time to save my results

In [90]:
ny_venues.to_csv('ny_venues.csv')

In [91]:
ny_venues = pd.read_csv('ny_venues.csv')

#### Let's take a look of summary info of the neighborhoods

In [92]:
neigh_list = list(ny_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside North York:')
print(len(neigh_list))
print('List of Neighborhoods inside North York:')
neigh_list

Number of Neighborhoods inside North York:
19
List of Neighborhoods inside North York:


['Hillcrest Village',
 'Henry Farm',
 'Bayview Village',
 'Silver Hills, York Mills',
 'Newtonbrook, Willowdale',
 'Willowdale West',
 'Parkwoods',
 'Flemingdon Park',
 'Bathurst Manor, Wilson Heights',
 'Northwood Park, York University',
 'CFB Toronto',
 'Downsview West',
 'Victoria Village',
 'Bedford Park',
 'Lawrence Heights, Lawrence Manor',
 'Glencairn',
 'Maple Leaf Park',
 'Humber Summit',
 'Emery, Humberlea']

In [93]:
neigh_venue_summary = ny_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
"Bathurst Manor, Wilson Heights",27,27,27,27,27,27,27
Bayview Village,13,13,13,13,13,13,13
Bedford Park,41,41,41,41,41,41,41
CFB Toronto,19,19,19,19,19,19,19
Downsview West,10,10,10,10,10,10,10


In [94]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(ny_venues['Venue Category'].unique())

There are 133 uniques categories.
Here is the list of different categories:


['Bakery',
 'Korean Restaurant',
 'Grocery Store',
 'Park',
 'Coffee Shop',
 'Bank',
 'Pizza Place',
 'Sandwich Place',
 'Pharmacy',
 'Housing Development',
 'Chinese Restaurant',
 'Ice Cream Shop',
 'Shopping Mall',
 'Recreation Center',
 'Residential Building (Apartment / Condo)',
 'Convenience Store',
 'Diner',
 'Toy / Game Store',
 'Electronics Store',
 'Tea Room',
 'Burger Joint',
 'Movie Theater',
 'Candy Store',
 'Fast Food Restaurant',
 'American Restaurant',
 'Department Store',
 'Smoothie Shop',
 'Salon / Barbershop',
 'Clothing Store',
 'Japanese Restaurant',
 'Caribbean Restaurant',
 'Theater',
 'Food Court',
 'Cosmetics Shop',
 'Liquor Store',
 'Restaurant',
 'Sporting Goods Shop',
 'Video Game Store',
 'Burrito Place',
 'Supermarket',
 'Beer Store',
 'Fried Chicken Joint',
 'Café',
 'Skating Rink',
 'Skate Park',
 'Pool',
 'Asian Restaurant',
 'Hookah Bar',
 'Dessert Shop',
 'Middle Eastern Restaurant',
 'Hot Dog Joint',
 'Indian Restaurant',
 'Ramen Restaurant',
 'Trail'

#### Now is time to make a one-hot Encode for unique features in columns

In [95]:
ny_onehot = pd.get_dummies(data = ny_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
ny_onehot.index = ny_onehot.index + 1
ny_onehot.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,Airport,American Restaurant,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,Bank,Bar,Beer Store,Bike Shop,Boutique,Bowling Alley,Bridal Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Fireworks Store,Fish & Chips Shop,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,History Museum,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
1,M2H,Hillcrest Village,43.803762,-79.363452,Tastee,This spot is popular,692,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M2H,Hillcrest Village,43.803762,-79.363452,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,This spot is popular,754,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M2H,Hillcrest Village,43.803762,-79.363452,Galati,This spot is popular,815,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M2H,Hillcrest Village,43.803762,-79.363452,Cummer Park,This spot is popular,776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,M2H,Hillcrest Village,43.803762,-79.363452,Tim Hortons,This spot is popular,731,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### I'm going to select only a couple of features to make the selector

In [98]:
ny_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'Bakery',
 'Korean Restaurant',
 'Pizza Place',
 'Sandwich Place',
 'Chinese Restaurant',
 'Diner',
 'Burger Joint',
 'Fast Food Restaurant',
 'American Restaurant',
 'Japanese Restaurant',
 'Caribbean Restaurant',
 'Restaurant',
 'Burrito Place',
 'Fried Chicken Joint',
 'Asian Restaurant',
 'Middle Eastern Restaurant',
 'Hot Dog Joint',
 'Indian Restaurant',
 'Ramen Restaurant',
 'Eastern European Restaurant',
 'Italian Restaurant',
 'Dim Sum Restaurant',
 'Sushi Restaurant',
 'Turkish Restaurant',
 'Vietnamese Restaurant',
 'Latin American Restaurant',
 'Portuguese Restaurant',
 'Wings Joint',
 'Comfort Food Restaurant',
 'Greek Restaurant',
 'Thai Restaurant',
 'Seafood Restaurant',
 'Mediterranean Restaurant',
 'Empanada Restaurant']

#### In the next cell I'm grouping into a DF

In [99]:
ny_onehot = ny_onehot[ny_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()

ny_onehot.head()

,Bakery,Korean Restaurant,Pizza Place,Sandwich Place,Chinese Restaurant,Diner,Burger Joint,Fast Food Restaurant,American Restaurant,Japanese Restaurant,Caribbean Restaurant,Food Court,Restaurant,Burrito Place,Fried Chicken Joint,Asian Restaurant,Middle Eastern Restaurant,Hot Dog Joint,Indian Restaurant,Ramen Restaurant,Eastern European Restaurant,Italian Restaurant,Dim Sum Restaurant,Sushi Restaurant,Turkish Restaurant,Vietnamese Restaurant,Latin American Restaurant,Portuguese Restaurant,Bagel Shop,Wings Joint,Comfort Food Restaurant,Greek Restaurant,Thai Restaurant,Seafood Restaurant,Mediterranean Restaurant,Empanada Restaurant
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Bathurst Manor, Wilson Heights",0,0,2,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Bayview Village,0,0,0,0,1,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bedford Park,1,0,1,1,0,0,0,3,1,1,0,0,1,0,0,0,0,0,1,0,0,3,0,1,0,0,0,0,1,1,1,1,1,0,0,0
CFB Toronto,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,2,1,1,0,0,0,0,0,0,0,0,0
Downsview West,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


#### Due to the size of the neighborhood data set, I did an integration of different restaurants, joints and places.

In [100]:
features_ny_list = list(ny_onehot.columns)
restaurant_list = []


for counter, value in enumerate(features_ny_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
ny_onehot['Total Restaurants'] = ny_onehot[restaurant_list].sum(axis = 1)
ny_onehot = ny_onehot.drop(columns = restaurant_list)


features_ny_list = list(ny_onehot.columns)
joint_list = []


for counter, value in enumerate(features_ny_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
ny_onehot['Total Joints'] = ny_onehot[joint_list].sum(axis = 1)
ny_onehot = ny_onehot.drop(columns = joint_list)

features_ny_list = list(ny_onehot.columns)
place_list = []

for counter, value in enumerate(features_ny_list):
    if value.find('Place') != (-1):
        place_list.append(value)
        
ny_onehot['Total Places'] = ny_onehot[place_list].sum(axis = 1)
ny_onehot = ny_onehot.drop(columns = place_list)

#### Once I have done the previous integration, It's time to see the results

In [101]:
ny_onehot

,Bakery,Diner,Food Court,Bagel Shop,Total Restaurants,Total Joints,Total Places
Neighborhood,,,,,,,
"Bathurst Manor, Wilson Heights",0,1,0,0,4,1,3
Bayview Village,0,0,0,0,4,0,0
Bedford Park,1,0,0,1,14,1,2
CFB Toronto,0,0,0,0,6,0,2
Downsview West,0,0,0,0,1,0,1
"Emery, Humberlea",1,0,0,0,0,0,0
Flemingdon Park,0,0,0,0,15,1,1
Glencairn,0,0,1,0,10,0,3
Henry Farm,2,0,1,0,7,2,4


#### To make my final analysis, I'm going to group into 5 cluster the previous results using K-Means

In [103]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 5, random_state = 0).fit(ny_onehot)

#### Finding the centers for each cluster

In [104]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = ny_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Diner,Food Court,Bagel Shop,Total Restaurants,Total Joints,Total Places,Total Sum
G5,0.5,0.000000,0.0,0.5,14.500000,1.000000,1.500000,18.000000
G2,0.5,0.000000,0.5,0.0,9.250000,1.750000,3.000000,15.000000
G3,0.0,0.333333,0.0,0.0,4.666667,0.333333,3.333333,8.666667
G4,0.6,0.200000,0.0,0.0,2.800000,0.000000,1.200000,4.800000
G1,0.4,0.000000,0.0,0.0,0.600000,0.000000,0.400000,1.400000


# Great!!!

***Watching the previous table we can say the next statements:***

## The Best Group is G5

### The Second Best Group is G2

#### The Third Best Group is G3

#### To make the result more understandable in the next cells we're goint to find the corresponding groups for each neighborhood

In [119]:
neigh_summary = pd.DataFrame([north_york.Neighbourhood, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
1,Hillcrest Village,4
2,Henry Farm,5
3,Bayview Village,3
4,"Silver Hills, York Mills",1
5,"Newtonbrook, Willowdale",1
6,Willowdale West,5
7,Parkwoods,2
8,Flemingdon Park,2
9,"Bathurst Manor, Wilson Heights",4
10,"Northwood Park, York University",4


# Analyzing results

## The Best Neighbourhood is/are

In [120]:
neigh_summary[neigh_summary['Group'] == 5]

,Neighborhood,Group
2,Henry Farm,5
6,Willowdale West,5


In [123]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 5]['Neighborhood'])[0]
ny_venues[ny_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M2J',
 'Neighborhood': 'Henry Farm',
 'Neighborhood Latitude': 43.7785175,
 'Neighborhood Longitude': -79.3465557}

## Second Best Neighbourhood is/are

In [121]:
neigh_summary[neigh_summary['Group'] == 2]

,Neighborhood,Group
7,Parkwoods,2
8,Flemingdon Park,2
11,CFB Toronto,2
13,Victoria Village,2


In [124]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 2]['Neighborhood'])[0]
ny_venues[ny_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M3A',
 'Neighborhood': 'Parkwoods',
 'Neighborhood Latitude': 43.7532586,
 'Neighborhood Longitude': -79.3296565}

## Third Best Neighbourhood is/are

In [122]:
neigh_summary[neigh_summary['Group'] == 3]

,Neighborhood,Group
3,Bayview Village,3
14,Bedford Park,3


In [125]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 3]['Neighborhood'])[0]
ny_venues[ny_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M2K',
 'Neighborhood': 'Bayview Village',
 'Neighborhood Latitude': 43.7869473,
 'Neighborhood Longitude': -79.385975}





# With this culminates this project, I want to thank you for reading until the end.

My name is Kevin Torres V.

Cheers!